### 참고 사이트
glove model : https://jxnjxn.tistory.com/49  
mecab : https://github.com/SOMJANG/Mecab-ko-for-Google-Colab  
mecab : https://velog.io/@kjyggg/  %ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EA%B8%B0-Mecab-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-A-to-Z%EC%84%A4%EC%B9%98%EB%B6%80%ED%84%B0-%EB%8B%A8%EC%96%B4-%EC%9A%B0%EC%84%A0%EC%88%9C%EC%9C%84-%EB%93%B1%EB%A1%9D%EA%B9%8C%EC%A7%80  
형태소 : https://mr-doosun.tistory.com/22

In [ ]:
# Install Wordcloud
!pip install wordcloud

from gensim.models import Word2Vec
from wordcloud import WordCloud, STOPWORDS
import gensim
import pandas as pd
from sklearn.manifold import TSNE
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from os import path
# from konlpy.tag import Komoran
import re
import sys
import datetime
%matplotlib inline 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# colab 환경에 glove를 설치해줍니다.
! pip install glove-python-binary

     |████████████████████████████████| 948 kB 5.1 MB/s 


In [ ]:
 !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [ ]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:
# 토크나이저 mecab 설치
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4 MB 4.7 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 448 kB 45.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-08-19 12:31:46--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c3:9b0a, 2406:da00:ff00::22c2:513, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=DYP%2BbXn1ZNyaM7pzGA9Bgy5MXkI%3D

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/welfare_tokenization/welfare_data.csv',encoding='utf8')
data.head()

,welfare_id,title,summary,who,criteria,what,how,calls,sites,category,dibs_count,last_update
0,607,고위험 임산부 의료비 지원,"고위험 임신의 치료와 관리에 필요한 진료비를 지원하여 경제적 부담을 줄이고, 건강한...","19대 고위험 임신질환(조기진통, 분만관련 출혈, 중증 임신중독증, 양막의 조기파열...",기초생활보장수급자 및 차상위계층이나 건강보험료 본인부담금 고지금액 기준으로 가구원수...,고위험 임산부 입원치료비의 급여 중 전액본인부담금 및 비급여 진료비의 90%(상급병...,시군구 보건소에 방문하여 신청합니다.,보건복지상담센터 129 전화걸기,보건복지상담센터 http://www.129.go.kr,0,NaN,2021-07-26 10:59:03
1,608,선천성대사이상 검사 및 환아관리,장애발생을 사전에 예방하여 영유아의 건강을 증진합니다.,선천성대사이상 검사비 지원 대상자는 기준중위소득 180%이하 가구의 영아입니다.,NaN,선천성대사이상 검사비 및 환아관리를 지원합니다.,보건소를 방문하여 신청합니다.,보건복지상담센터 129 전화걸기,보건복지상담센터 http://www.129.go.kr,0,NaN,2021-07-26 10:59:03
2,609,고용보험 미적용자 출산급여 지원,소득활동을 하고 있으나 고용보험의 '출산전후휴가급여'를 지원받지 못하는 출산여성에게...,소득활동을 하고 있지만 고용보험 미적용으로 '출산전후휴가급여'를 받지 못하는 출산여...,"출산전 18개월중 3개월 이상, 그리고 출산일 현재도 소득활동을 하고 있어야 하며 ...",고용보험 미적용 출산급여 지원 대상자에게 다음과 같이 지원됩니다.\n\n출산급여 :...,"고용보험 홈페이지(www.ei.go.kr)에 회원 가입 후 신청하거나, 가까운 고용...",고용노동부 고객상담센터 1350 전화걸기,고용보험 http://www.ei.go.kr,0,NaN,2021-07-26 10:59:03
3,610,입양숙려기간 모자지원사업,미혼 또는 이혼 한부모가 출산 전후 정서적으로 불안정한 상태로 입양에 동의하는 것을...,지원기간 동안 입양에 동의한 사실이 없는 출산 예정 또는 출산한 미혼/이혼 한부모를...,혼인관계에 있지 않은 출산(예정)일 전 40일 또는 출산 후 7일 이내의 혼인관계증...,아래의 서비스 이용비용을 지원합니다.\n\n가정 내 산후지원인력의 가정방문서비스 1...,관할 시/군/구청을 방문해서 신청합니다.,보건복지상담센터 129 전화걸기,보건복지상담센터 http://www.129.go.kr,0,NaN,2021-07-26 10:59:03
4,611,아동통합서비스지원(드림스타트사업),취약계층 아동에게 맞춤형 통합서비스를 제공하여 건강한 성장과 발달을 도모하고 공평한...,0세(임산부)이상 만 12세(초등학생 이하) 취약계층 아동 및 가족이 대상입니다.\...,"사업지역에 거주하는 해당 연령 아동과 그 가족, 임산부(0세)를 대상으로 아동 양육...",가정 방문을 통해 서비스 대상 아동을 발굴하여 통합적으로 사례를 관리하는 기본 서비...,시·군·구청에 방문하거나 전화 또는 우편 및 인터넷 등으로 신청합니다.,보건복지상담센터 129 전화걸기,보건복지부 http://www.mohw.go.kr,0,NaN,2021-07-26 10:59:03


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   welfare_id   356 non-null    int64  
 1   title        356 non-null    object 
 2   summary      356 non-null    object 
 3   who          356 non-null    object 
 4   criteria     235 non-null    object 
 5   what         352 non-null    object 
 6   how          355 non-null    object 
 7   calls        355 non-null    object 
 8   sites        330 non-null    object 
 9   category     356 non-null    int64  
 10  dibs_count   0 non-null      float64
 11  last_update  356 non-null    object 
dtypes: float64(1), int64(2), object(9)
memory usage: 33.5+ KB


In [ ]:
welfare_data=data[['title', 'summary']]
welfare_data.head()


,title,summary
0,고위험 임산부 의료비 지원,"고위험 임신의 치료와 관리에 필요한 진료비를 지원하여 경제적 부담을 줄이고, 건강한..."
1,선천성대사이상 검사 및 환아관리,장애발생을 사전에 예방하여 영유아의 건강을 증진합니다.
2,고용보험 미적용자 출산급여 지원,소득활동을 하고 있으나 고용보험의 '출산전후휴가급여'를 지원받지 못하는 출산여성에게...
3,입양숙려기간 모자지원사업,미혼 또는 이혼 한부모가 출산 전후 정서적으로 불안정한 상태로 입양에 동의하는 것을...
4,아동통합서비스지원(드림스타트사업),취약계층 아동에게 맞춤형 통합서비스를 제공하여 건강한 성장과 발달을 도모하고 공평한...


In [ ]:
len(welfare_data)

356

In [ ]:
import re

def preprocess(x):
    # 전화번호 제거
    text = re.sub(r'[0-9]{2,3}(-[0-9]{3,4}){2}','',x) 
    # 숫자 3개 이상일 경우 제거
    text = re.sub(r'[0-9]{3,}','',text)
    # 특수 문자 제거
    text = re.sub(r'[^\s0-9a-zA-Z가-힣:]','',text)
    text = ' '.join([i.upper() for i in text.split() if len(i) >= 1])
    return text

# data['처리내역'] = data['처리내역'].astype(str)
welfare_data['title'] = welfare_data['title'].apply(lambda x: preprocess(x))
welfare_data['summary'] = welfare_data['summary'].apply(lambda x: preprocess(x))
#welfare_data = welfare_data[welfare_data.title != ''].reset_index(drop=True)
#welfare_data = welfare_data[welfare_data.summary != ''].reset_index(drop=True)
len(welfare_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


356

In [ ]:
welfare_data.head()

,title,summary
0,고위험 임산부 의료비 지원,고위험 임신의 치료와 관리에 필요한 진료비를 지원하여 경제적 부담을 줄이고 건강한 ...
1,선천성대사이상 검사 및 환아관리,장애발생을 사전에 예방하여 영유아의 건강을 증진합니다
2,고용보험 미적용자 출산급여 지원,소득활동을 하고 있으나 고용보험의 출산전후휴가급여를 지원받지 못하는 출산여성에게 출...
3,입양숙려기간 모자지원사업,미혼 또는 이혼 한부모가 출산 전후 정서적으로 불안정한 상태로 입양에 동의하는 것을...
4,아동통합서비스지원드림스타트사업,취약계층 아동에게 맞춤형 통합서비스를 제공하여 건강한 성장과 발달을 도모하고 공평한...


In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

text = '고위험 임산부 의료비 지원		'
nouns = mecab.nouns(text) 
morphs = mecab.morphs(text)
pos = mecab.pos(text)  
print(nouns)
print(morphs)
print(pos)

['고위험', '임산부', '의료비', '지원']
['고위험', '임산부', '의료비', '지원']
[('고위험', 'NNP'), ('임산부', 'NNG'), ('의료비', 'NNG'), ('지원', 'NNG')]


### 사용자 사전을 만들고, 불용어, 품사 지정해서 제거하는 preprocessing 해주어야 함

### word vector 학습하기

In [ ]:
welfare_data['title_token']=welfare_data['title'].apply(lambda x: mecab.morphs(x))
welfare_data['summary_token']=welfare_data['summary'].apply(lambda x: mecab.morphs(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
welfare_data.head()

,title,summary,title_token,summary_token
0,고위험 임산부 의료비 지원,고위험 임신의 치료와 관리에 필요한 진료비를 지원하여 경제적 부담을 줄이고 건강한 ...,"[고위험, 임산부, 의료비, 지원]","[고위험, 임신, 의, 치료, 와, 관리, 에, 필요, 한, 진료비, 를, 지원, ..."
1,선천성대사이상 검사 및 환아관리,장애발생을 사전에 예방하여 영유아의 건강을 증진합니다,"[선천성, 대사, 이상, 검사, 및, 환, 아, 관리]","[장애, 발생, 을, 사전, 에, 예방, 하, 여, 영, 유아, 의, 건강, 을, ..."
2,고용보험 미적용자 출산급여 지원,소득활동을 하고 있으나 고용보험의 출산전후휴가급여를 지원받지 못하는 출산여성에게 출...,"[고용, 보험, 미, 적, 용자, 출산, 급여, 지원]","[소득, 활동, 을, 하, 고, 있, 으나, 고용, 보험, 의, 출산, 전후, 휴가..."
3,입양숙려기간 모자지원사업,미혼 또는 이혼 한부모가 출산 전후 정서적으로 불안정한 상태로 입양에 동의하는 것을...,"[입양, 숙려, 기간, 모자, 지원, 사업]","[미혼, 또는, 이혼, 한, 부모, 가, 출산, 전후, 정서, 적, 으로, 불, 안..."
4,아동통합서비스지원드림스타트사업,취약계층 아동에게 맞춤형 통합서비스를 제공하여 건강한 성장과 발달을 도모하고 공평한...,"[아동, 통합, 서비스, 지원, 드림, 스타트, 사업]","[취약, 계층, 아동, 에게, 맞춤, 형, 통합, 서비스, 를, 제공, 하, 여, ..."


In [ ]:
title_token_list=list(welfare_data['title_token'])
summary_token_list=list(welfare_data['summary_token'])
sent_token=title_token_list+summary_token_list

In [ ]:
sent_token

[['고위험', '임산부', '의료비', '지원'],
 ['선천성', '대사', '이상', '검사', '및', '환', '아', '관리'],
 ['고용', '보험', '미', '적', '용자', '출산', '급여', '지원'],
 ['입양', '숙려', '기간', '모자', '지원', '사업'],
 ['아동', '통합', '서비스', '지원', '드림', '스타트', '사업'],
 ['의료', '급여', '본인', '부담', '면제'],
 ['난임', '부부', '시술', '비', '지원'],
 ['미숙아', '및', '선천성', '이', '상아', '의료비', '지원'],
 ['지역', '사회', '통합', '건강', '증진', '사업'],
 ['인플루엔자', '국가', '예방', '접종', '지원', '사업'],
 ['여성', '장애', '인', '출산', '비용', '지원'],
 ['의료', '급여', '선택', '의료', '급여', '기관', '제'],
 ['산모', '신생아', '건강', '관리', '지원', '사업'],
 ['모성',
  '보호',
  '육아',
  '지원',
  '출산',
  '전후',
  '휴가',
  '유산',
  '사',
  '산휴',
  '가',
  '포함',
  '급여',
  '육아',
  '휴직',
  '등',
  '급여'],
 ['에너지', '바우처'],
 ['청소년', '산모', '임신', '출산', '의료비', '지원'],
 ['선천성', '난청', '검사', '및', '보청기', '지원'],
 ['저소득층', '기저귀', '조제분유', '지원'],
 ['의료', '급여'],
 ['어촌', '생활', '돌봄', '지원'],
 ['긴급', '복지', '해산', '비', '지원'],
 ['의료', '급여', '임신', '출산', '진료비', '지원'],
 ['출산', '육', '아기', '고용', '안정', '장려금'],
 ['다', '문화', '가족', '방문', '교육', '서비스'],
 ['유아', '학비', '보

In [ ]:
# glove import 해서 불러옵니다.
from glove import Corpus, Glove

In [ ]:
DATA_DIR = '/content/drive/MyDrive/welfare_tokenization' 

In [ ]:
# corpus 생성
corpus = Corpus()
corpus.fit(sent_token, window=20)

In [ ]:
# model
glove = Glove(no_components=32, learning_rate=0.01)     # 0.05
%time glove.fit(corpus.matrix, epochs=10, no_threads=4, verbose=False)    # Wall time: 8min 32s
glove.add_dictionary(corpus.dictionary)

# save
glove.save(DATA_DIR + '/glove_w20_epoch10.model')

CPU times: user 619 ms, sys: 257 µs, total: 619 ms
Wall time: 336 ms


In [ ]:
# load glove
glove_model = Glove.load(DATA_DIR + '/glove_w20_epoch10.model')

In [ ]:
glove_model.word_vectors

array([[ 0.00125331, -0.01054825, -0.01299009, ...,  0.00573277,
        -0.01133097, -0.00208074],
       [ 0.01489642,  0.0076859 , -0.00409738, ...,  0.01055649,
        -0.00457415,  0.00062872],
       [ 0.00496536, -0.0091485 ,  0.00740463, ...,  0.00416115,
         0.01242429,  0.01580399],
       ...,
       [-0.0044282 , -0.0095573 ,  0.01012027, ...,  0.00347425,
         0.01499382,  0.01284051],
       [ 0.00463893,  0.00394785,  0.00562408, ...,  0.00604958,
        -0.00536271,  0.00877904],
       [ 0.0049048 ,  0.00411781,  0.00363938, ...,  0.01378862,
        -0.01356374,  0.00901789]])

In [ ]:
# word dict 생성
word_dict = {}
for word in  glove_model.dictionary.keys():
    word_dict[word] = glove_model.word_vectors[glove_model.dictionary[word]]
print('[Success !] Lengh of word dict... : ', len(word_dict))

# save word_dict
import pickle
with open(DATA_DIR + '/glove_word_dict_64.pickle', 'wb') as f:
    pickle.dump(word_dict, f)
print('[Success !] Save word dict!...')

[Success !] Lengh of word dict... :  1541
[Success !] Save word dict!...


In [ ]:
word_dict['임산부']

array([ 0.01489642,  0.0076859 , -0.00409738, -0.00431306, -0.00417101,
       -0.00431187, -0.00532354,  0.00726073, -0.00020332, -0.00990588,
       -0.00903397,  0.01182329,  0.01040684,  0.00475249, -0.01260338,
        0.0093457 ,  0.01185445, -0.00198647,  0.01258015, -0.01537758,
       -0.00040762,  0.00428515,  0.00856914, -0.01234821, -0.00454914,
        0.01032241,  0.00702797,  0.00878889,  0.00776161,  0.01055649,
       -0.00457415,  0.00062872])

In [ ]:
# test data(질의) 속 word dict에 없는 단어는 64차원 0벡터로 임베딩시킴
# word dict : train데이터의 임베딩 사전
total_word_dict = {}
cnt = 0

for word in test_tokens:
    if word in word_dict.keys():
        total_word_dict[word] = word_dict[word]
    else:
        word_dict[word] = np.zeros((128))
        cnt += 1  # 처음 본 단어 갯수 세기

print('no train word -> 0....', cnt)
print('token -> word embedding....!',len(unique_tokens))

In [ ]:
# 워드 임베딩된 벡터로 문장 단위 임베딩
def sent2vec_glove(tokens, embedding_dim=32):
    '''문장 token 리스트를 받아서 임베딩 시킨다.'''
  
    size = len(tokens)
    matrix = np.zeros((size, embedding_dim))
    word_table = word_dict     # glove word_dict

    for i, token in enumerate(tokens):
        vector = np.array([
            word_table[t] for t in token
            if t in word_table
        ])

        if vector.size != 0:
            final_vector = np.mean(vector, axis=0)
            matrix[i] = final_vector

    return matrix

In [ ]:
# 문장 임베딩
sentence_tokens = sent_token
sentence_glove = sent2vec_glove(sentence_tokens)
sentence_glove.shape

(712, 32)

In [ ]:
# clustering
k = 16
kmeans = KMeans(n_clusters=k, random_state=2021)
y_pred = kmeans.fit_predict(sentence_glove)

# tsne
tsne = TSNE(verbose=1, perplexity=100, random_state=2021)     # perplexity : 유사정도
X_embedded = tsne.fit_transform(sentence_glove)
print('Embedding shape 확인', X_embedded.shape)

# 시각화
sns.set(rc={'figure.figsize':(20,20)})
# colors
palette = sns.hls_palette(16, l=.4, s=.9) 
# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y_pred,
                legend='full',palette=palette)     # kmeans로 예측

plt.title('t-SNE with KMeans Labels and Glove Embedding')
#plt.savefig(DATA_DIR + "/t-sne_question_glove_embedding.png")
plt.show()

In [ ]:
### sentence embedding 한 것을 문서랑 맵핑해서 질의 sentence와 가장 유사한 것을 찾으면 됨.

In [ ]:
# 코사인 유사도를 구하는 함수
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [ ]:
sentence_glove.shape #712문장의 32벡터

In [ ]:
sentence_glove

array([[ 0.0069199 , -0.0204201 ,  0.00740835, ..., -0.00120182,
        -0.00940111,  0.00296225],
       [-0.00423507, -0.00883821,  0.00036671, ...,  0.00263861,
         0.00434799,  0.00296878],
       [-0.00243354, -0.0052097 ,  0.00414206, ..., -0.0005456 ,
        -0.00655342, -0.00431854],
       ...,
       [-0.00351418, -0.02023918,  0.00795393, ..., -0.00612944,
        -0.00760908, -0.00588601],
       [-0.00078575, -0.02164818,  0.01175487, ..., -0.00353503,
        -0.00803448,  0.00196197],
       [-0.00408757, -0.0085037 ,  0.00707132, ..., -0.00043988,
        -0.0022784 , -0.00200362]])

In [ ]:
# 고위험 임산부 의료비 지원과 가장 유사도가 높은 문서를 구해보자.
# 임산부 복지정책에 대해 알려줘-> "임산부"
# "임산부", "고위험", "지원"

In [ ]:
sample_query = sent2vec_glove([['임산부']])

In [ ]:
sample_query[0,:]

array([ 0.01489642,  0.0076859 , -0.00409738, -0.00431306, -0.00417101,
       -0.00431187, -0.00532354,  0.00726073, -0.00020332, -0.00990588,
       -0.00903397,  0.01182329,  0.01040684,  0.00475249, -0.01260338,
        0.0093457 ,  0.01185445, -0.00198647,  0.01258015, -0.01537758,
       -0.00040762,  0.00428515,  0.00856914, -0.01234821, -0.00454914,
        0.01032241,  0.00702797,  0.00878889,  0.00776161,  0.01055649,
       -0.00457415,  0.00062872])

In [ ]:
sample_query

array([[ 0.01489642,  0.0076859 , -0.00409738, -0.00431306, -0.00417101,
        -0.00431187, -0.00532354,  0.00726073, -0.00020332, -0.00990588,
        -0.00903397,  0.01182329,  0.01040684,  0.00475249, -0.01260338,
         0.0093457 ,  0.01185445, -0.00198647,  0.01258015, -0.01537758,
        -0.00040762,  0.00428515,  0.00856914, -0.01234821, -0.00454914,
         0.01032241,  0.00702797,  0.00878889,  0.00776161,  0.01055649,
        -0.00457415,  0.00062872]])

In [ ]:
cos_sim_query = pd.DataFrame()

In [ ]:
for i in range(0,712):
  cos_sim_query.loc[i, 'cos_sim'] = cos_sim(sentence_glove[i,:], sample_query[0,:])


In [ ]:
cos_sim_query.sort_values(by=['cos_sim'], axis=0, ascending=False)

In [ ]:
data.iloc[0, :]

In [ ]:
data.iloc[31, :]

In [ ]:
data.iloc[246, :]

In [ ]:
data.iloc[241, :]